In [34]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.nn import functional as F

In [35]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [36]:
!nvidia-smi

Sun Jul  4 01:18:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.30       Driver Version: 462.30       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 2060   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   53C    P8    12W /  N/A |   1353MiB /  6144MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [37]:
import wandb
%env "WANDB_NOTEBOOK_NAME" "demo_wine_wandb_test"
wandb.login()

env: "WANDB_NOTEBOOK_NAME"="demo_wine_wandb_test"


True

In [38]:
df = pd.read_csv("./data/wine_data.csv")

In [39]:
df.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,0,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,0,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,0,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,0,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [40]:
df.shape

(178, 14)

In [41]:
df.describe()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,0.938202,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.775035,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,0.000000,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,0.000000,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,1.000000,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,2.000000,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,2.000000,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


# DATA WRANGLING

check for nulls and duplicates

In [42]:
df.isna().sum()

Class                           0
Alcohol                         0
Malic acid                      0
Ash                             0
Alcalinity of ash               0
Magnesium                       0
Total phenols                   0
Flavanoids                      0
Nonflavanoid phenols            0
Proanthocyanins                 0
Color intensity                 0
Hue                             0
OD280/OD315 of diluted wines    0
Proline                         0
dtype: int64

In [43]:
df.duplicated().sum()

0

# MACHINE LEARNING
### ML PREP

In [44]:
# Encode target labels with value between 0 and n_classes-1.

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report


In [45]:
le = LabelEncoder()
df['Class'] = le.fit_transform(df['Class'])
df.sample(10)

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
26,0,13.39,1.77,2.62,16.1,93,2.85,2.94,0.34,1.45,4.80,0.92,3.22,1195
55,0,13.56,1.73,2.46,20.5,116,2.96,2.78,0.20,2.45,6.25,0.98,3.03,1120
151,2,12.79,2.67,2.48,22.0,112,1.48,1.36,0.24,1.26,10.80,0.48,1.47,480
172,2,14.16,2.51,2.48,20.0,91,1.68,0.70,0.44,1.24,9.70,0.62,1.71,660
24,0,13.50,1.81,2.61,20.0,96,2.53,2.61,0.28,1.66,3.52,1.12,3.82,845
100,1,12.08,2.08,1.70,17.5,97,2.23,2.17,0.26,1.40,3.30,1.27,2.96,710
165,2,13.73,4.36,2.26,22.5,88,1.28,0.47,0.52,1.15,6.62,0.78,1.75,520
107,1,12.72,1.75,2.28,22.5,84,1.38,1.76,0.48,1.63,3.30,0.88,2.42,488
44,0,13.05,1.77,2.10,17.0,107,3.00,3.00,0.28,2.03,5.04,0.88,3.35,885
67,1,12.37,1.17,1.92,19.6,78,2.11,2.00,0.27,1.04,4.68,1.12,3.48,510


In [46]:
df['Class'].unique()

array([0, 1, 2], dtype=int64)

### SEPARATE FETURES AND TARGET

In [47]:
df_features = df.drop('Class', axis=1)
df_features.head()

,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [48]:
df_target = df[['Class']]
df_target.head()

,Class
0,0
1,0
2,0
3,0
4,0


In [49]:
#SPLIT DATA
from sklearn.model_selection import train_test_split

In [50]:
X_train, x_test, Y_train, y_test = train_test_split(df_features, 
                                                    df_target,
                                                    test_size=0.3,
                                                     random_state=42)

In [51]:
X_train.shape, x_test.shape,

((124, 13), (54, 13))

In [52]:
 Y_train.shape, y_test.shape

((124, 1), (54, 1))

### Convert data to Tensors for Pytorch

In [53]:
Xtrain = torch.from_numpy(X_train.values).float()
Xtest = torch.from_numpy(x_test.values).float()
print(Xtrain.shape, Xtest.shape)

torch.Size([124, 13]) torch.Size([54, 13])


In [54]:
print(Xtrain.dtype, Xtest.dtype)

torch.float32 torch.float32


We have successfully converted our  X_data into torch tensors of float32 datatype

In [55]:
Ytrain = torch.from_numpy(Y_train.values).view(1,-1)[0]
Ytest = torch.from_numpy(y_test.values).view(1, -1)[0]
print(Ytrain.shape, Ytest.shape)

torch.Size([124]) torch.Size([54])


We use the **view()** to reshape the tensor.<br>
The loss function doesn't support multi-target and therefore, we should use a 1D Tensor of 1 row containing the labels.<br>
We have successfully converted our y_data

In [56]:
print(Ytrain.dtype, Ytest.dtype)

torch.int64 torch.int64


## PyTorch
### We create a classifier and define our neural network for our model

### Hyperparameterss

In [57]:
input_size = 13
output_size = 3
hidden_size = 100

In [58]:
config = dict(
                input_size = 13,
                output_size = 3,
                hidden_size = 100,
                dataset = "wine dataset",
                architecture = 'Linear', 
                learning_rate = 0.01,
                loss = nn.NLLLoss(),
)


In [59]:
for k,v in config.items():
    print(k, v)

input_size 13
output_size 3
hidden_size 100
dataset wine dataset
architecture Linear
learning_rate 0.01
loss NLLLoss()


### Define the neural network


In [60]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, X):
        X = torch.sigmoid((self.fc1(X)))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

In [61]:
model = Net()
# preview out model
model

Net(
  (fc1): Linear(in_features=13, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=3, bias=True)
)

### Define Optimizer and Loss Function

In [62]:
import torch.optim as optim

In [63]:
optimizer = optim.Adam(model.parameters(), lr=config.get("learning_rate"))
loss_fn = config.get("loss")

In [64]:
epochs = 1000
with wandb.init(project="demo_wandb_test", config = config):
    wandb.watch(model, criterion=None, log="gradients", log_freq=10)
    
    for epoch in range(epochs):
        
        optimizer.zero_grad()
        Ypred = model(Xtrain)

        loss = loss_fn(Ypred, Ytrain)
        loss.backward()

        optimizer.step()

        wandb.log({'Epoch': epoch, "Loss": loss.item(), "Predicted Values" : Ypred.values})


    # SAVE MODEL STATE DICT TO DISK

    wandb.save(torch.save(model.state_dict(), "./models/home_state_dict.pt"))

    # LOAD MODEL FROM DISK and EVALUATE
    new_model =  Net()
    new_model.load_state_dict(torch.load("./models/home_state_dict.pt"))
    new_model.eval()

    # SET THE PREDICTIONS
    predict = new_model(Xtest)
    _, predict_y = torch.max(predict, 1)

    # VISUALIZE CONFUSION MATRIX

    wandb.sklearn.plot_confusion_matrix(Ytest, predict_y, labels = [0,1,2])
    # Print Metrics
    wandb.log({"accuracy_score" : accuracy_score(Ytest, predict_y),
    "precision_score" : precision_score(Ytest, predict_y, average='weighted'),
    "recall_score": recall_score(Ytest, predict_y, average="weighted")})
wandb.finish()


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Epoch,999
Loss,0.00044
Predicted Values,None.Tensor.values
_runtime,11
_timestamp,1625350743
_step,1001
accuracy_score,0.88889
precision_score,0.89509
recall_score,0.88889


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Loss,█▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▃▃▃▃▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆███
_timestamp,▁▁▁▃▃▃▃▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
accuracy_score,▁
precision_score,▁
recall_score,▁


TypeError: 'Summary' object is not callable